In [49]:
import pandas as pd
from src.preprocessing import process_composites

parents = 1

df_data = pd.read_csv("data/processed/1_parents/total.tsv", sep="\t", dtype={"code": str})
df_vars = pd.read_csv("data/variables/processed/variables.tsv", sep="\t", dtype={"code": str})
df_data = process_composites(df_data, df_vars)


In [52]:

ls_found_codes = list(set(df_data["code"].unique()).intersection(set(df_vars["code"].unique())))
var_codes = df_vars["code"].unique()

print(f"Found {len(ls_found_codes)} codes in the data")
df_data["parent"] = df_data["code_wp"].apply(lambda x: set(eval(x)).intersection(set(var_codes)))
df_data["parent_matches"] = df_data["parent"].apply(lambda x: len(x))
df_data.loc[(df_data["parent_matches"] > 0)&(~df_data["code"].isin(var_codes)), "semantic_rel"] = "PARENT"
df_data.loc[(df_data["parent_matches"] > 0)&(~df_data["code"].isin(var_codes)), "code"] = df_data.loc[(df_data["parent_matches"] > 0)&(~df_data["code"].isin(var_codes)), "parent"].apply(lambda x: x[0])

# df_data_f = df_data[df_data["code"].isin(ls_found_codes)].copy()

# df_data_nf = df_data[~ df_data["code"].isin(ls_found_codes)].copy()
# df_data_nf["parent"] = df_data_nf["code_wp"].apply(lambda x: set(eval(x)).intersection(set(var_codes)))
# df_data_nf["parent_matches"] = df_data_nf["parent"].apply(lambda x: len(x))


# var_codes = df_vars["code"].unique()
# df_data_nf["parent"] = df_data_nf["code_wp"].apply(lambda x: set(eval(x)).intersection(set(var_codes)))
# df_data_nf["parent_matches"] = df_data_nf["parent"].apply(lambda x: len(x))
# df_data_nf_parents = df_data_nf[df_data_nf["parent_matches"] > 0].copy()
# df_data_nf_parents["semantic_rel"] = "PARENT"
# df_data_nf_nparents = df_data_nf[df_data_nf["parent_matches"] == 0].copy()


Found 133 codes in the data


In [54]:
df_data[df_data["semantic_rel"] == "PARENT"]

,filename,label,off0,off1,span,code,semantic_rel,code_wp,code_list,var_code,parent,parent_matches
13,es-S1137-66272006000500012-1,ENFERMEDAD,1052,1072,hepatoesplenomegalia,80515008,PARENT,"['16294009', '80515008']",NaN,NaN,{80515008},1
15,es-S1130-63432016000100009-1,ENFERMEDAD,98,113,ictus isquémico,230690007,PARENT,"['230690007', '389100007', '609410002', '87536...",NaN,NaN,{230690007},1
51,es-S1130-63432013000300012-1,ENFERMEDAD,139,158,valvulopatía mitral,368009,PARENT,"['301101006', '368009']",NaN,NaN,{368009},1
81,es-S1130-05582015000300004-1,ENFERMEDAD,345,375,hipertensión arterial esencial,38341003,PARENT,['38341003'],NaN,NaN,{38341003},1
128,es-S0376-78922011000200004-1,ENFERMEDAD,53,80,insuficiencia renal crónica,709044004,PARENT,"['42399005', '709044004']",NaN,NaN,{709044004},1
...,...,...,...,...,...,...,...,...,...,...,...,...
30115,S1137-66272011000300024-1,FARMACO,1519,1524,c-LDL,22244007,PARENT,"['22244007', '84698008']",NaN,NaN,"{22244007, 84698008}",2
30749,S0212-71992005000200008-1,FARMACO,174,180,Hb 1AC,38082009,PARENT,['38082009'],NaN,NaN,{38082009},1
31750,S1137-66272006000500011-1,FARMACO,1095,1098,LDL,22244007,PARENT,"['22244007', '84698008']",NaN,NaN,"{22244007, 84698008}",2
33049,S1139-76322015000100010-1,FARMACO,821,844,hemoglobina glicosilada,38082009,PARENT,['38082009'],NaN,NaN,{38082009},1


In [41]:
df_data.rename(columns={"label": "label_corpus"}, inplace=True)
df_code_ovr = df_vars[["ID", "name", "code", "term", "label"]].merge(
                                                            df_data[['code', 'span', 'semantic_rel', "label_corpus"]].reset_index(drop=True), 
                                                            on='code', 
                                                            how='left',
                                                        ).fillna("NOT_FOUND")



# df_code_ovr = pd.concat([df_code_ovr, df_data_nf_parents, df_data_nf_nparents], ignore_index=True, axis=0)
# df_data_nf[df_data_nf["parent_matches"].apply(lambda x: len(x) > 0)]
# df_data_nf["code_wp"] = df_data_nf["code_wp"].apply(lambda x: eval(x))
# df_data_nf = df_data_nf.explode(column="code_wp")
# df_data_nf["old_code"] = df_data_nf["code"]
# df_data_nf["code"] = df_data_nf["code_wp"]

# df_code_ovr_nf = df_vars[["ID", "name", "code", "term", "label"]].merge(
#                                                                             df_data_nf[['code', 'span', 'semantic_rel', "label_corpus", "old_code"]].reset_index(), 
#                                                                             on='code',
#                                                                             how='left',
#                                                                         ).fillna("NOT_FOUND")

# df_code_ovr_nf_found = df_code_ovr_nf[df_code_ovr_nf["span"] != "NOT_FOUND"].copy()
# df_code_ovr_nf_found['semantic_rel'] = "PARENT"
# ls_fixed_codes = df_code_ovr_nf_found["old_code"].unique()
# df_code_ovr_nf_found.drop(columns=["old_code"], inplace=True)
# df_code_ovr = df_code_ovr[~df_code_ovr["code"].isin(ls_fixed_codes)]
# df_code_ovr = pd.concat([df_code_ovr, df_code_ovr_nf_found], ignore_index=True)

# df_code_ovr = df_code_ovr.groupby(['ID', 'name', 'code', 'span', 'term', 'semantic_rel', 'label', 'label_corpus']).size().reset_index(name='count').sort_values(by="count", ascending=False)
# df_code_ovr = df_code_ovr.sort_values(by=['ID', 'name', 'span', 'term', 'semantic_rel', 'label', 'label_corpus', "count"], ascending=False)
# # df_code_count = df_code_ovr.drop_duplicates(subset=['ID', 'name', 'span', 'term', 'semantic_rel', 'label', 'label_corpus'])

print(df_code_ovr.head(50).to_markdown())


|    | ID          | name                          |       code | term                                      | label             | span                                    | semantic_rel   | label_corpus   |
|---:|:------------|:------------------------------|-----------:|:------------------------------------------|:------------------|:----------------------------------------|:---------------|:---------------|
|  0 | DT4HDem-3   | Sex female                    |  248152002 | Female                                    | Human             | NOT_FOUND                               | NOT_FOUND      | NOT_FOUND      |
|  1 | DT4HDem-4   | Sex male                      |  248153007 | Male                                      | Human             | NOT_FOUND                               | NOT_FOUND      | NOT_FOUND      |
|  2 | DT4HDem-6   | YOB                           |  184099003 | Date of birth                             | other             | NOT_FOUND                               | NOT_

In [42]:
df_data_nf_parents

,filename,label_corpus,off0,off1,span,code,semantic_rel,code_wp,code_list,var_code,parent,parent_matches
13,es-S1137-66272006000500012-1,ENFERMEDAD,1052,1072,hepatoesplenomegalia,36760000,PARENT,"['16294009', '80515008']",NaN,NaN,{80515008},1
15,es-S1130-63432016000100009-1,ENFERMEDAD,98,113,ictus isquémico,422504002,PARENT,"['230690007', '389100007', '609410002', '87536...",NaN,NaN,{230690007},1
51,es-S1130-63432013000300012-1,ENFERMEDAD,139,158,valvulopatía mitral,11851006,PARENT,"['301101006', '368009']",NaN,NaN,{368009},1
81,es-S1130-05582015000300004-1,ENFERMEDAD,345,375,hipertensión arterial esencial,59621000,PARENT,['38341003'],NaN,NaN,{38341003},1
128,es-S0376-78922011000200004-1,ENFERMEDAD,53,80,insuficiencia renal crónica,90688005,PARENT,"['42399005', '709044004']",NaN,NaN,{709044004},1
...,...,...,...,...,...,...,...,...,...,...,...,...
30115,S1137-66272011000300024-1,FARMACO,1519,1524,c-LDL,102739008,PARENT,"['22244007', '84698008']",NaN,NaN,"{22244007, 84698008}",2
30749,S0212-71992005000200008-1,FARMACO,174,180,Hb 1AC,259689004,PARENT,['38082009'],NaN,NaN,{38082009},1
31750,S1137-66272006000500011-1,FARMACO,1095,1098,LDL,102739008,PARENT,"['22244007', '84698008']",NaN,NaN,"{22244007, 84698008}",2
33049,S1139-76322015000100010-1,FARMACO,821,844,hemoglobina glicosilada,733829007,PARENT,['38082009'],NaN,NaN,{38082009},1


In [ ]:

if parents > 0:
    ls_found_codes = df_code_ovr[df_code_ovr["span"] != "NOT_FOUND"].code.unique()
    df_data_nf = df_data[~ df_data["code"].isin(ls_found_codes)].copy()
    df_data_nf = df_data_nf.explode(column="code_wp")
    df_data_nf["old_code"] = df_data_nf["code"]
    df_data_nf["code"] = df_data_nf["code_wp"]

    df_code_ovr_nf = df_vars[["ID", "name", "code", "term", "label"]].merge(
                                                                                df_data_nf[['code', 'span', 'semantic_rel', "label_corpus", "old_code"]].reset_index(), 
                                                                                on='code',
                                                                                how='left',
                                                                            ).fillna("NOT_FOUND")
    df_code_ovr_nf_found = df_code_ovr_nf[df_code_ovr_nf["span"] != "NOT_FOUND"].copy()
    df_code_ovr_nf_found['semantic_rel'] = "PARENT"
    ls_fixed_codes = df_code_ovr_nf_found["old_code"].unique()
    df_code_ovr_nf_found.drop(columns=["old_code"], inplace=True)
    df_code_ovr = df_code_ovr[~df_code_ovr["code"].isin(ls_fixed_codes)]
    df_code_ovr = pd.concat([df_code_ovr, df_code_ovr_nf_found], ignore_index=True)
    
    print()